In [1]:
!pip install pandas pyvis


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   --------------------------------------- 756.0/756.0 kB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 22.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pyvis.network import Network
from datetime import datetime

# Load your LinkedIn connections CSV
df = pd.read_csv("Connections.csv", quotechar='"', sep=',', engine='python')

# Create a PyVis network
net = Network(height="850px", width="100%", bgcolor="#0b0c10", font_color="white", notebook=False)

# Add yourself as the central node
net.add_node("You", color="gold", size=40, title="Sumwon Special")

# Optional: clean up date and company fields
df['Connected On'] = pd.to_datetime(df['Connected On'], errors='coerce')

# Generate company color palette (assign a unique color per company)
companies = df['Company'].fillna("Unknown").unique()
color_palette = {}
colors = [
    "#66b2ff", "#a3e4d7", "#f7dc6f", "#f1948a", "#bb8fce", "#73c6b6",
    "#f8c471", "#7fb3d5", "#82e0aa", "#f5b7b1", "#5dade2", "#af7ac5"
]
for i, company in enumerate(companies):
    color_palette[company] = colors[i % len(colors)]

# Add nodes for each LinkedIn connection
for _, row in df.iterrows():
    name = f"{row['First Name']} {row['Last Name']}"
    company = row['Company'] if pd.notna(row['Company']) else "Unknown"
    position = row['Position'] if pd.notna(row['Position']) else "No title listed"
    connected_on = row['Connected On'].strftime("%b %Y") if pd.notna(row['Connected On']) else "Unknown date"

    net.add_node(
        name,
        title=f"{position}<br>{company}<br>Connected: {connected_on}",
        color=color_palette[company],
        size=15
    )

    # Add an edge between you and your connection
    net.add_edge("You", name)

# Optionally, connect people who work at the same company
for company in companies:
    people = df[df['Company'] == company]['First Name'] + " " + df[df['Company'] == company]['Last Name']
    people = people.tolist()
    for i in range(len(people)):
        for j in range(i + 1, len(people)):
            net.add_edge(people[i], people[j], color="#444444")

# Set physics layout
net.toggle_physics(True)

# Save and open as interactive HTML
net.write_html("linkedin_connections_network.html", open_browser=True)
